In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import gym
import itertools
import matplotlib
import numpy as np
import sys
import sklearn.pipeline
import sklearn.preprocessing
import pickle
import matplotlib.pyplot as plt

from estimator import Estimator
from value_estimator import ValueEstimator

from sklearn.linear_model import SGDRegressor
from sklearn.kernel_approximation import RBFSampler

matplotlib.style.use('ggplot')


In [2]:
def parse(s):
    split = s.split(',')
    n = np.zeros(len(split))
    for i in range(len(split)):
        n[i] = float(split[i])
    return n

In [4]:
num_observations = 1268140
with open("demonstrations_10k") as f:
    s_arr = np.zeros((num_observations, 11))
    r_arr = np.zeros((num_observations, 1))
    a_arr = np.zeros((num_observations, 3))
    sprime_arr = np.zeros((num_observations, 11))
    
    i = 0

    for l in f:
        if len(l.strip().split(';')) <= 1:
            continue
    
        i += 1
        s, r, a, sp = l.strip().split(';')
        s_arr[i, :] = parse(s)
        r_arr[i, :] = parse(r)
        a_arr[i, :] = parse(a)
        sprime_arr[i, :] = parse(sp)


In [ ]:
# Feature Preprocessing: Normalize to zero mean and unit variance
# We use a few samples from the observation space to do this
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(s_arr)

# Used to convert a state to a featurizes represenation.
# We use RBF kernels with different variances to cover different parts of the space
featurizer = sklearn.pipeline.FeatureUnion([
        ("rbf1", RBFSampler(gamma=5.0, n_components=100)),
        ("rbf2", RBFSampler(gamma=2.0, n_components=100)),
        ("rbf3", RBFSampler(gamma=1.0, n_components=100)),
        ("rbf4", RBFSampler(gamma=0.5, n_components=100))
        ])
featurizer.fit(scaler.transform(s_arr))